# Final Motion Detector Component

In [ ]:
# Python program to implement 
# Webcam Motion Detector
  
# importing OpenCV, time and Pandas library
import cv2, time, pandas
# importing datetime class from datetime library
from datetime import datetime

# Capturing video
video = cv2.VideoCapture(0)

# Assigning our static_back to None
static_back = None
static_datetime = None
static_small_centroid = None
DELAY = 0.500
CONTOUR_RECTANGLE = 10000

# Infinite while loop to treat stack of image as video
deadline = time.time()
movement_direction = None
while time.time() - deadline < DELAY and  movement_direction == None:
    # Reading frame(image) from video
    check, frame = video.read()
  
    # Converting color image to gray_scale image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  
    # Converting gray scale image to GaussianBlur 
    # so that change can be find easily
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
  
    # In first iteration we assign the value 
    # of static_back to our first frame
    if static_back is None:
        static_back = gray
        static_datetime =time.time()
        continue
  
    # Difference between static background 
    # and current frame(which is GaussianBlur)
    diff_frame = cv2.absdiff(static_back, gray)
    
    
    # If change in between static background and
    # current frame is greater than 30 it will show white color(255)
    thresh_frame = cv2.threshold(diff_frame, 30, 255, cv2.THRESH_BINARY)[1]
    thresh_frame = cv2.dilate(thresh_frame, None, iterations = 2)
  
    # Finding contour of moving object
    cnts,_ = cv2.findContours(thresh_frame.copy(), 
                       cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    centroid = []
    area = []
    for contour in cnts:
        if cv2.contourArea(contour) < CONTOUR_RECTANGLE:
            continue
        (x, y, w, h) = cv2.boundingRect(contour)
        # making green rectangle around the moving object
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
        #Getting the centroid & area of each contour
        centroid.append([x+w/2, y+h/2])
        area.append(w*h)
    #Getting index of contour with less area
    
    if len(area) > 0:
        index = area.index(min(area))
        del area
        small_centroid = centroid[index]
        del centroid

        # In first iteration we assign the value 
        # of static_small_centroid to our first frame
        if static_small_centroid is None:
            static_small_centroid = small_centroid
            static_datetime =time.time()
            continue
        else:
            #Checking direction after delay
            if time.time() - static_datetime > DELAY:
                horizontal_right = small_centroid[0] - static_small_centroid[0] < 0
                vertical_up = small_centroid[1] - static_small_centroid[1] < 0
                movement_direction = [horizontal_right, vertical_up]
                static_small_centroid = small_centroid
                static_datetime =time.time()
                static_back = gray
  
    # Displaying image in gray_scale
    cv2.imshow("Gray Frame", gray)
  
    # Displaying the difference in currentframe to
    # the staticframe(very first_frame)
    cv2.imshow("Difference Frame", diff_frame)
  
    # Displaying the black and white image in which if
    # intensity difference greater than 30 it will appear white
    cv2.imshow("Threshold Frame", thresh_frame)
  
    # Displaying color frame with contour of motion of object
    cv2.imshow("Color Frame", frame)
  
    key = cv2.waitKey(1)
    # if q entered whole process will stop
    if key == ord('q'):
        break
    
print ("movement_direction: ", movement_direction)
  
video.release()
  
# Destroying all the windows
cv2.destroyAllWindows()

# Motion Detector Component Coupled with Video Game

In [ ]:
import cv2
import pygame
from src.components.game_status import GameStatus
from src.config import Config
from src.game_phases import main_menu_phase, gameplay_phase, exit_game_phase
from src.global_state import GlobalState
from src.services.music_service import MusicService

pygame.init()
FramePerSec = pygame.time.Clock()

DELAY = 0.021 #0.034 #0.100
CONTOUR_RECTANGLE = 3400 #5000 #10000
MINIMUM_CONTOUR_TYPE = False


def update_game_display():
    pygame.display.update()
    FramePerSec.tick(Config.FPS)

def main():

    # Capturing video
    video = cv2.VideoCapture(0)
    
    #Temporal static variables
    static_back = None
    static_datetime = None
    static_small_centroid = None
    
    while True:
        
        if GlobalState.GAME_STATE == GameStatus.MAIN_MENU:
            main_menu_phase()
            
        elif GlobalState.GAME_STATE == GameStatus.GAMEPLAY:
            static_back, static_datetime, static_small_centroid = gameplay_phase(video, 
                                                                                 static_back, 
                                                                                 static_datetime, 
                                                                                 static_small_centroid,
                                                                                 DELAY, 
                                                                                 CONTOUR_RECTANGLE, 
                                                                                 MINIMUM_CONTOUR_TYPE
                                                                                )
        elif GlobalState.GAME_STATE == GameStatus.GAME_END:
            exit_game_phase()

        MusicService.start_background_music()
        update_game_display()

if __name__ == "__main__":
    main()